<H1> 19BCE060 IRS PRAC3

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
from collections import Counter
from nltk import word_tokenize

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

tf

In [ ]:
def tf(question, word):
    if word not in question:
        return 0
    count = dict(Counter(question))
    q_len = len(question)
    return float(count[word]) / float(q_len)

In [ ]:
def n_containing(qlist, word):
    return float(qlist[word])

def idf(qlist, word):
    return math.log(float(len(qlist.keys())) / (1.0 + n_containing(qlist, word)))

In [ ]:
def tfidf(question, qlist, word):
    return tf(question, word) * idf(qlist, word)

In [ ]:
def cosine(v1, v2):
    v1 = np.array(v1)
    v2 = np.array(v2)

    return np.dot(v1, v2) / (np.sqrt(np.sum(v1**2)) * np.sqrt(np.sum(v2**2)))

In [ ]:
train = pd.read_csv('../input/quora-question-pairs/train.csv.zip')
train_qs = train[['id', 'question1', 'question2', 'is_duplicate']]
qlist = []
count = 0
for row in train_qs.itertuples():
    try:
        if len(str(row[2])) > 10:
            q1 = word_tokenize(row[2].lower())
        if len(str(row[3])) > 10:
            q2 = word_tokenize(row[3].lower())
        qlist += q1 + q2
        count+=1
        if count%100000 == 0:
            print('At'+str(count))
#        qlist.append(q2)
    except TypeError:
        pass
qlist = dict(Counter(qlist))
import json
with open('qlist.json', 'w') as f:
    f.write(json.dumps(qlist, indent=2))
print('All Questions added to list')

In [ ]:
with open('submission.csv', 'a') as f:
    f.write('id,is_duplicate\n')
for row in train_qs.itertuples():
    if len(str(row[2])) > 10 and len(str(row[3])) > 10:
        wordvec1 = word_tokenize(row[2].lower())
        wordvec2 = word_tokenize(row[3].lower())
        words = wordvec1 + wordvec2
        words = list(set([word for word in words if word != '?']))

        # print words

        vec1 = []
        vec2 = []
        for word in words:
            vec1.append(tfidf(wordvec1, qlist, word))
            vec2.append(tfidf(wordvec2, qlist, word))

        with open('submission.csv', 'a') as f:
            f.write(str(row[1]) + "," + str(cosine(vec1, vec2)) + '\n')
    else:
        with open('submission.csv', 'a') as f:
            f.write(str(row[1]) + "," + '0' + '\n')
#    print str(row[1]) + "," + str(cosine(vec1, vec2))